In [1]:
#Recreate correct results from Investigate Ordering of Columns.Ipynb 

In [2]:
#Sktime imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sktime.classification.shapelet_based import MrSEQLClassifier
from sklearn.model_selection import GroupShuffleSplit

from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
import sys
from os.path import exists

import importlib

#EEG package modules_____________________________________________________________________________

import constants
import custom_ts_length
# import eeg_stat_ts
import format_eeg_data
import run_expts

#Load the appropriate data ________________________________________________________________________
'''
o 1. PD vs HC,   2.(PD , PD+RBD) vs HC,   3.(PD + PD&RBD) vs HC vs RBD  multiclass, 4.PD vs PD+RBD vs RBD vs HC 
'''
core_path = '/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/'
folder = 'REM_full_expts/'
#Load the appropriate X (full set of features ) , y (full set of labels) , groups 
load_path = core_path + folder
X = pd.read_hdf(load_path + 'X.h5', key='df', mode='r')
y = pd.read_hdf(load_path + 'y_full.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + 'groups.h5', key='df', mode='r')  

clfs = {'MrSQL_1' : MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) }

subset_names_and_cols = run_expts.gen_subsets_dict()

In [13]:
subset_regions = {}
for reg in list(subset_names_and_cols.keys())[:3]:
    subset_regions[reg] = subset_names_and_cols[reg]

print(subset_regions)

{'Prefrontal': ['Delta_Prefrontal', 'Theta_Prefrontal', 'Alpha_Prefrontal', 'Sigma_Prefrontal', 'Beta_Prefrontal', 'Gamma_Prefrontal', 'TotalAbsPow_Prefrontal'], 'Frontal': ['Delta_Frontal', 'Theta_Frontal', 'Alpha_Frontal', 'Sigma_Frontal', 'Beta_Frontal', 'Gamma_Frontal', 'TotalAbsPow_Frontal'], 'Left Frontal': ['Delta_Left Frontal', 'Theta_Left Frontal', 'Alpha_Left Frontal', 'Sigma_Left Frontal', 'Beta_Left Frontal', 'Gamma_Left Frontal', 'TotalAbsPow_Left Frontal']}


In [4]:
power_band_cols = ['Delta', 'Theta', 'Alpha', 'Sigma', 'Beta', 'Gamma', 'TotalAbsPow']

subsets_dict_new = {}
for col in power_band_cols:
    name = col + '_' + 'Prefrontal'
    subsets_dict_new[name] = [name]

subsets_dict_new
# subsets_reversed

delta_only = {'Delta_Prefrontal': ['Delta_Prefrontal']}

theta_only = {'Theta_Prefrontal': ['Theta_Prefrontal']}

In [14]:
results_dict = {}
# for num in [1,2,3,4]:
for num in [1]:
    print('commencing for ' + str(num) + '...')

    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups,num)

    t1 = time.time()
    results_df = run_expts.run_mv_tsc(X_expt ,y_expt , groups_expt, clfs , return_df = True, subset_names_and_cols = subset_regions, random_states = [13 , 37])
    
    t2 = time.time()
    
    results_dict[num] = results_df
    
print(t2-t1)

commencing for 1...
378.24321246147156


In [15]:
results_dict[num]

,random_state,y_true,Prefrontal_MrSQL_1_y_preds,Frontal_MrSQL_1_y_preds,Left Frontal_MrSQL_1_y_preds
0,13,"[[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...","[[1, 0, 0, 1, 0, 0, 1, 0], [1, 1, 0, 0, 0, 0, ..."
1,37,"[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ...","[[0, 1, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, ...","[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, ..."


In [16]:
df = run_expts.generate_subset_acc_std( results_dict[num], return_df = True)

In [17]:
df

,random_state,Prefrontal_MrSQL_1_acc,Prefrontal_MrSQL_1_std,Frontal_MrSQL_1_acc,Frontal_MrSQL_1_std,Left Frontal_MrSQL_1_acc,Left Frontal_MrSQL_1_std
0,13,0.666746,0.196194,0.524286,0.148880,0.350476,0.077055
1,37,0.617222,0.166744,0.532778,0.160174,0.463889,0.060604


In [8]:
df

,random_state,Theta_Prefrontal_MrSQL_1_acc,Theta_Prefrontal_MrSQL_1_std
0,13,0.925000,0.150000
1,37,0.935556,0.052775
